# Ejercicio: Mejorar un modelo de regresión logística
En el ejercicio anterior, ajustamos un modelo de regresión logística simple para predecir la probabilidad de una avalancha. Esta vez, mejoraremos su rendimiento utilizando múltiples características de forma inteligente.

## Visualización de datos
Vamos a cargar nuestros datos.

In [ ]:
!pip install statsmodels

In [1]:
import pandas

!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/avalanche.csv
import graphing # custom graphing code. See our GitHub repo for details

#Import the data from the .csv file
dataset = pandas.read_csv('avalanche.csv', delimiter="\t", index_col=0)

# Split our data into training and test
import sklearn.model_selection
train, test = sklearn.model_selection.train_test_split(dataset, test_size=0.25, random_state=10)

print("Train size:", train.shape[0])
print("Test size:", test.shape[0])

#Let's have a look at the data
print(train.head())

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


Train size: 821
Test size: 274
     avalanche  no_visitors  surface_hoar  fresh_thickness  wind  weak_layers  \
176          0            9      5.142447         9.877195     6            8   
114          1            3      8.170281         9.136835    34            7   
869          0            3      1.979579         9.497017    10            8   
775          1            0      1.999078         9.337908    21            6   
181          1            9      6.854401         6.099359    22            5   

     tracked_out  
176            1  
114            0  
869            0  
775            0  
181            0  


Disponemos de numerosas funciones:

+ surface_hoar es lo alterada que está la superficie de la nieve 
+ espesor_fresco es el espesor de la capa superior de nieve, o 0 si no hay nieve fresca encima
+ viento es la velocidad máxima del viento ese día, en km/h
+ capas_débiles es el número de capas de nieve que no están bien unidas a otras capas
+ no_visitors es el número de excursionistas que estaban en el sendero ese día
+ tracked_out es un 1 o un 0. Un 1 significa que la nieve ha sido muy pisoteada por los excursionistas.

## Regresión logística simple
Hagamos un modelo de regresión logística simple y evaluemos su rendimiento con precisión.

In [2]:
import sklearn
from sklearn.metrics import accuracy_score
import statsmodels.formula.api as smf

# Perform logistic regression.
model = smf.logit("avalanche ~ weak_layers", train).fit()

# Calculate accuracy
def calculate_accuracy(model):
    '''
    Calculates accuracy
    '''
    # Make estimations and convert to categories
    avalanche_predicted = model.predict(test) > 0.5

    # Calculate what proportion were predicted correctly
    # We can use sklearn to calculate accuracy for us
    print("Accuracy:", accuracy_score(test.avalanche, avalanche_predicted))

calculate_accuracy(model)

Optimization terminated successfully.
         Current function value: 0.616312
         Iterations 5
Accuracy: 0.6167883211678832


Veamos cómo podemos mejorar nuestro modelo.
## Utilizando múltiples características 
La mayoría de nuestras funciones parecen ser útiles, al menos en teoría. Probemos un modelo con todas las características que tenemos disponibles.

In [3]:
# Perform logistic regression.
model_all_features = smf.logit("avalanche ~ weak_layers + surface_hoar + fresh_thickness + wind + no_visitors + tracked_out", train).fit()
calculate_accuracy(model_all_features)

Optimization terminated successfully.
         Current function value: 0.459347
         Iterations 7
Accuracy: 0.7846715328467153


Esa es una gran mejora con respecto al modelo más simple con el que hemos estado trabajando. Para entender por qué, podemos mirar la información resumida

In [4]:
model_all_features.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              avalanche   No. Observations:                  821
Model:                          Logit   Df Residuals:                      814
Method:                           MLE   Df Model:                            6
Date:                Sat, 25 Mar 2023   Pseudo R-squ.:                  0.3305
Time:                        13:06:02   Log-Likelihood:                -377.12
converged:                       True   LL-Null:                       -563.33
Covariance Type:            nonrobust   LLR p-value:                 2.372e-77
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -4.0107      0.443     -9.043      0.000      -4.880      -3.141
weak_layers         0.3733      0.034     10.871      0.000       0.306       0.441
surface_hoar        0.3306      0.035      9.424      0.000       0.262       0.399
fresh_thickness    -0.0220      0.030     -0.732      0.464      -0.081       0.037
wind                0.1009      0.009     11.149      0.000       0.083       0.119
no_visitors        -0.1060      0.032     -3.262      0.001      -0.170      -0.042
tracked_out        -0.0664      0.181     -0.367      0.713      -0.420       0.288
===================================================================================
"""

Echa un vistazo a la columna P, recordando que los valores inferiores a 0,05 significan que podemos confiar en que este parámetro está ayudando al modelo a hacer mejores predicciones.

Tanto surface_hoar como wind tienen valores muy pequeños aquí, lo que significa que son predictores útiles y probablemente explican por qué nuestro modelo funciona mejor. Si nos fijamos en la columna coef (que indica los parámetros) vemos que tienen valores positivos. Esto significa que los vientos más fuertes y la mayor cantidad de hielo en superficie aumentan el riesgo de aludes.

## Simplificando nuestro modelo
Si volvemos a mirar el resumen, vemos que tracked_out (el grado de pisado de la nieve) y fresh_thickness tienen valores p elevados. Esto significa que no son predictores útiles. Veamos qué ocurre si los eliminamos de nuestro modelo:

In [5]:
# Perform logistic regression.
model_simplified = smf.logit("avalanche ~ weak_layers + surface_hoar + wind + no_visitors", train).fit()
calculate_accuracy(model_simplified)

Optimization terminated successfully.
         Current function value: 0.459760
         Iterations 7
Accuracy: 0.781021897810219


Nuestro nuevo modelo funciona de forma muy similar al anterior. En algunas circunstancias, simplificar un modelo de este tipo puede incluso mejorarlo, ya que es menos probable que se ajuste en exceso.

## Selección cuidadosa de las características
Normalmente, no elegimos las características a ciegas. Pensemos en lo que acabamos de hacer: hemos eliminado la cantidad de nieve fresca de un modelo que trataba de predecir avalanchas. Algo parece fuera de lugar. ¿Seguramente las avalanchas son mucho más probables después de que ha nevado? Del mismo modo, el número de personas en la pista no parece estar relacionado con el número de avalanchas, pero sabemos que las personas a menudo pueden desencadenar avalanchas.

Repasemos de nuevo nuestro modelo anterior:

In [6]:
model_all_features.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              avalanche   No. Observations:                  821
Model:                          Logit   Df Residuals:                      814
Method:                           MLE   Df Model:                            6
Date:                Sat, 25 Mar 2023   Pseudo R-squ.:                  0.3305
Time:                        13:07:29   Log-Likelihood:                -377.12
converged:                       True   LL-Null:                       -563.33
Covariance Type:            nonrobust   LLR p-value:                 2.372e-77
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -4.0107      0.443     -9.043      0.000      -4.880      -3.141
weak_layers         0.3733      0.034     10.871      0.000       0.306       0.441
surface_hoar        0.3306      0.035      9.424      0.000       0.262       0.399
fresh_thickness    -0.0220      0.030     -0.732      0.464      -0.081       0.037
wind                0.1009      0.009     11.149      0.000       0.083       0.119
no_visitors        -0.1060      0.032     -3.262      0.001      -0.170      -0.042
tracked_out        -0.0664      0.181     -0.367      0.713      -0.420       0.288
===================================================================================
"""

Mira la fila fresh_thickness. Nos dice que tiene un coeficiente negativo. Esto significa que a medida que aumenta el espesor, disminuyen las avalanchas.

Del mismo modo, no_visitors tiene un coeficiente negativo, lo que significa que menos excursionistas significan más avalanchas.

¿Cómo puede ser esto? Bueno, mientras que los visitantes pueden provocar aludes si hay mucha nieve fresca, presumiblemente no pueden hacerlo fácilmente si no hay nieve fresca. Esto significa que nuestras características no son totalmente independientes.

Podemos decirle al modelo que intente tener en cuenta que estas características interactúan, utilizando un signo multiplicador. Intentémoslo ahora.

In [7]:
# Create a model with an interaction. Notice the end of the string where
# we've a multiply sign between no_visitors and fresh_thickness
formula = "avalanche ~ weak_layers + surface_hoar + wind + no_visitors * fresh_thickness"
model_with_interaction = smf.logit(formula, train).fit()
calculate_accuracy(model_with_interaction)

Optimization terminated successfully.
         Current function value: 0.413538
         Iterations 7
Accuracy: 0.8357664233576643


¡El modelo ha mejorado hasta un 84 % de precisión! Veamos la información resumida:

In [8]:
model_with_interaction.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              avalanche   No. Observations:                  821
Model:                          Logit   Df Residuals:                      814
Method:                           MLE   Df Model:                            6
Date:                Sat, 25 Mar 2023   Pseudo R-squ.:                  0.3973
Time:                        13:08:29   Log-Likelihood:                -339.51
converged:                       True   LL-Null:                       -563.33
Covariance Type:            nonrobust   LLR p-value:                 1.587e-93
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -0.9606      0.587     -1.636      0.102      -2.111       0.190
weak_layers                     0.4327      0.039     11.193      0.000       0.357       0.508
surface_hoar                    0.3887      0.039     10.035      0.000       0.313       0.465
wind                            0.1204      0.010     11.607      0.000       0.100       0.141
no_visitors                    -0.9430      0.114     -8.237      0.000      -1.167      -0.719
fresh_thickness                -0.4962      0.069     -7.191      0.000      -0.631      -0.361
no_visitors:fresh_thickness     0.1015      0.013      7.835      0.000       0.076       0.127
===============================================================================================
"""

Podemos ver que el término de interacción es útil: el valor p es inferior a 0,05. El modelo también funciona mejor que nuestros intentos anteriores.

## Predicciones con múltiples características

Muy rápidamente, vamos a explorar lo que significa esta interacción observando las predicciones del modelo. 

Primero graficaremos dos características independientes en 3D. Empezaremos con "capas_débiles" y "viento":

In [9]:
graphing.model_to_surface_plot(model_with_interaction, ["weak_layers", "wind"], test)

Creating plot...


El gráfico es interactivo: gírelo y explore cómo existe una relación clara en forma de S entre las características y la probabilidad. Veamos ahora las características que hemos dicho que pueden interactuar

In [10]:
graphing.model_to_surface_plot(model_with_interaction, ["no_visitors", "fresh_thickness"], test)

Creating plot...


Su aspecto es muy diferente. Desde cualquier lado, podemos ver una forma de s, pero éstas se combinan de formas extrañas.

Podemos ver que el riesgo aumenta los días con muchos visitantes y mucha nieve. No hay riesgo real de avalancha cuando hay mucha nieve pero no hay visitantes, o cuando hay muchos visitantes pero no hay nieve.

El hecho de que muestre un riesgo alto cuando no hay nieve fresca ni visitantes podría deberse a la lluvia, que mantiene alejados a los visitantes y las nubes de nieve pero provoca aludes de la nieve más antigua. Para confirmar esto, necesitaríamos explorar los datos más a fondo, pero nos detendremos aquí por ahora.

Resumen
Bien hecho. Recapitulemos. Hemos

mejorado nuestro modelo simple añadiendo más características.
practicado la interpretación de los coeficientes (parámetros) de nuestro modelo a partir del resumen del modelo
eliminado características innecesarias
explorado cómo a veces es importante pensar en lo que realmente significan los datos
hemos creado un modelo que combina características para obtener un resultado superior